In [1]:
from pathlib import Path

import pytz

from post_processing.utils.fpod_utils import csv_folder, pod2aplose, actual_data, meta_cut_aplose
from post_processing.utils.core_utils import json2df

### Load data
🐬 = input to modify \
Import your raw FPOD or CPOD data. All files for one site must be stored in the same folder and identified by their respective phases. \
You also need to import your metadata file.

In [2]:
pod_files = Path(r"U:\CA4") #Path to your data folder. 🐬
path = csv_folder(pod_files) #Process all your POD.csv files.
json = Path(r"C:\Users\fouinel\Downloads\deployment_calais.json") #Path to your metadata file. 🐬

print(path.head())
df_0 = path.dropna()

metadatax = json2df(json_path=json)
metadatax["deploy.name"] = (metadatax["site.name"].astype(str) + "_" +  metadatax["campaign.name"].astype(str))

                      File          ChunkEnd  DPM    Nall  MinsOn deploy.name
0  CA4  POD2397 file01.CP3  14/05/2014 07:07  0.0     0.0     1.0  CA4_Phase1
1  CA4  POD2397 file01.CP3  14/05/2014 07:08  0.0     8.0     1.0  CA4_Phase1
2  CA4  POD2397 file01.CP3  14/05/2014 07:09  0.0     4.0     1.0  CA4_Phase1
3  CA4  POD2397 file01.CP3  14/05/2014 07:10  0.0   251.0     1.0  CA4_Phase1
4  CA4  POD2397 file01.CP3  14/05/2014 07:11  0.0  4095.0     1.0  CA4_Phase1


In [ ]:
d_beg_end = actual_data(df_0, metadatax) #Extract the beginning and end of recording for every phase.

In [3]:
df_1 = df_0[df_0["DPM"] !=0 ] #Remove the 0 to lighten the APLOSE file.

### APLOSE format
#### *C-POD*
Use cpod2aplose if you are managing C-POD data.

In [4]:
df_aplose = pod2aplose(df_1, pytz.utc, "CA4", "Marsouin", "CPOD") #Precise site name, species and instrument. 🐬
df_aplose["deploy.name"] = df_aplose["filename"]
print(df_aplose.head())

  dataset    filename  start_time  end_time  start_frequency  end_frequency  \
0     CA4  CA4_Phase1           0        60                0              0   
1     CA4  CA4_Phase1           0        60                0              0   
2     CA4  CA4_Phase1           0        60                0              0   
3     CA4  CA4_Phase1           0        60                0              0   
4     CA4  CA4_Phase1           0        60                0              0   

  annotation annotator                start_datetime  \
0   Marsouin      CPOD  2014-05-17T03:52:00.000+0000   
1   Marsouin      CPOD  2014-05-17T04:47:00.000+0000   
2   Marsouin      CPOD  2014-05-19T17:06:00.000+0000   
3   Marsouin      CPOD  2014-05-20T11:07:00.000+0000   
4   Marsouin      CPOD  2014-05-20T11:16:00.000+0000   

                   end_datetime  is_box deploy.name  
0  2014-05-17T03:53:00.000+0000       0  CA4_Phase1  
1  2014-05-17T04:48:00.000+0000       0  CA4_Phase1  
2  2014-05-19T17:07:00.000

### Clean your dataset
Remove useless lines, recorder outside the instrument submersion. Export your file to the aplose format. You can change the name of the file to match the project you are working on.

In [5]:
cleared = meta_cut_aplose(df_aplose, metadatax)

### Export

In [ ]:
d_beg_end.to_csv(r"U:\Deb_Fin_CA4.csv", index=False) #Export the new file. 🐬
cleared.to_csv(r"U:\APLOSE_CA4_pos.csv", index=False) #Name your file. 🐬